In [29]:
"""Drag roi and save data."""

import cv2
import os
import scanpy as sc
import json
from shapely.geometry import Point, Polygon

In [43]:
cell = sc.read_h5ad('/home/tom/github/niceview/db/data/gt-iz-p9-rep2-cell.h5ad')
with open('/home/tom/github/niceview/user/test.json', 'r') as f:
    coords = json.load(f)
img_path = '/home/tom/github/niceview/db/data/gt-iz-p9-rep2-wsi-img.tiff'

In [ ]:
def get_bounding_box(coords):
    x_coords, y_coords = zip(*coords)
    x1, y1 = min(x_coords), min(y_coords)
    x2, y2 = max(x_coords), max(y_coords)
    return x1, y1, x2, y2

In [52]:
def get_roi(coords, adata, img, home_dir):
    """Get roi from coordinates."""
    for idx, coord in enumerate(coords):
        # save adata
        roi = Polygon(coord)
        locs = list(map(lambda x: roi.contains(Point(x)), adata.obsm['spatial']))
        to_keep = adata[locs].copy()
        h5ad_path = os.path.join(home_dir, f'roi-{idx}.h5ad')
        to_keep.write_h5ad(h5ad_path)
        
        # save image
        x1, y1, x2, y2 = get_bounding_box(coord)
        cropped_region = img[y1:y2, x1:x2]
        cv2.imwrite(os.path.join(home_dir, f'roi-{idx}.tiff'), cropped_region)

In [53]:
get_roi(coords, cell, img_path, '/home/tom/github/niceview/user')

: 